In [1]:
!pip install -r requirements.txt

In [2]:
import json
import pandas as pd

In [3]:
def ensure_utf8_encoding(data):
  if isinstance(data, str):
    return data.encode('utf-8', 'replace').decode('utf-8')
  elif isinstance(data, list):
    return [ensure_utf8_encoding(item) for item in data]
  elif isinstance(data, dict):
    return {key: ensure_utf8_encoding(value) for key, value in data.items()}
  else:
    return data

with open("/content/raw_posts.json", encoding='utf-8') as f:
  posts = json.load(f)
posts = ensure_utf8_encoding(posts)

In [4]:
for post in posts:
  print(post)

{'text': "Just saw a LinkedIn Influencer with 'Organic Growth' written in the profile with 65K+ followers claiming that he can help you in growing your platform, copying the posts from other influencers.", 'engagement': 90}
{'text': "Jobseekers, this one’s for you.\n Every application, every interview, every follow-up… the pressure is immense.\n And I know what you're thinking: Am I not good enough? \n But let me tell you, this isn’t about you or your skills. It’s about a broken system where 60% of applicants never hear back. \n Your mental health is not worth sacrificing for a system that doesn’t acknowledge your worth. \n Please remember, taking care of yourself is the real priority. \n Your dream job will come, but for now, breathe. 🌻", 'engagement': 347}
{'text': 'Looking for jobs on LinkedIn is like online dating: Full of promises, but in the end, you’re just left ghosted.', 'engagement': 109}
{'text': "LinkedIn scams be like: 'Congratulations, you've been selected for a role you 

In [5]:
from google.colab import userdata
GROQ_API_KEY = userdata.get('GROQ_API_KEY')

In [6]:
from langchain_groq import ChatGroq
llm = ChatGroq(groq_api_key = GROQ_API_KEY, model_name = 'llama-3.3-70b-versatile')

In [7]:
# Checking LLM activation
if __name__ == "__main__":
  response = llm.invoke("which are top 3 pillars of a country's economy. Share in less than 50 words")
  print(response.content)


The top 3 pillars of a country's economy are: 
1. Industry
2. Services
3. Agriculture


In [8]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.exceptions import OutputParserException
json_parser = JsonOutputParser()

In [9]:
def process_posts(raw_file_path, processed_file_path=None):
  enriched_posts = []
  for post in posts:
    mdata = metadata(post['text'])
    post_with_metadata = post | mdata
    enriched_posts.append(post_with_metadata)
    unified_tags = get_unified_tags(enriched_posts)
    for post in enriched_posts:
        current_tags = post['tags']
        new_tags = {unified_tags[tag] for tag in current_tags if tag in unified_tags}
        post['tags'] = list(new_tags)
    with open(processed_file_path, encoding='utf-8', mode="w") as outfile:
      json.dump(enriched_posts, outfile, indent=4)

In [10]:
def metadata(post):
  template = '''
    You are given a LinkedIn post. You need to extract number of lines, language of the post and tags.
    1. Return a valid JSON. No preamble.
    2. JSON object should have exactly three keys: line_count, language and tags.
    3. tags is an array of text tags. Extract maximum two tags.
    4. Each tag should be enclosed in double quotes.
    5. Language should be English or Hinglish (Hinglish means hindi + english)

    Here is the actual post on which you need to perform this task:
    {post}
    '''
  prompttemplate = PromptTemplate.from_template(template)
  chain = prompttemplate|llm
  response = chain.invoke(input = {"post":post})
  try:
    res = json_parser.parse(response.content)
  except OutputParserException:
    raise OutputParserException("1. LLM Returned Code instead of JSON OR Context too big. Unable to Parso jobs")
  return res

In [11]:
def get_unified_tags(posts_with_metadata):
    unique_tags = set()
    # Loop through each post and extract the tags
    for post in posts_with_metadata:
        unique_tags.update(post['tags'])  # Add the tags to the set

    unique_tags_list = ','.join(unique_tags)

    template = '''I will give you a list of tags. You need to unify tags with the following requirements,
    1. Tags are unified and merged to create a shorter list.
       Example 1: "Jobseekers", "Job Hunting" can be all merged into a single tag "Job Search".
       Example 2: "Motivation", "Inspiration", "Drive" can be mapped to "Motivation"
       Example 3: "Personal Growth", "Personal Development", "Self Improvement" can be mapped to "Self Improvement"
       Example 4: "Scam Alert", "Job Scam" etc. can be mapped to "Scams"
       Example 5: "sapna" can be mapped to "Mental Health"
    2. Each tag should be follow title case convention. example: "Motivation", "Job Search"
    3. Output should be a strictly a JSON object, No preamble.
    3. Output should have mapping of original tag and the unified tag.
       For example: {{"Jobseekers": "Job Search",  "Job Hunting": "Job Search", "Motivation": "Motivation}}
       Here is the list of tags:
        {tags}
       '''
    pt = PromptTemplate.from_template(template)
    chain = pt | llm
    response = chain.invoke(input={"tags": str(unique_tags_list)})
    try:
      json_parser = JsonOutputParser()
      res = json_parser.parse(response.content)
    except OutputParserException:
      raise OutputParserException("2.Context too big. Unable to parse jobs.")
    return res

In [12]:
if __name__ == "__main__":
  process_posts("/content/raw_posts.json","/content/drive/MyDrive/Linkedin_Post_Generator/Processed_posts.json")


In [16]:
with open('app.py', 'w') as f:
    f.write("""
import streamlit as st
from few_shots import FewShotPosts
from post_generator import generate_post
fs = FewShotPosts()
tags = fs.get_tags()


length_options = ["Short", "Medium", "Long"]
language_options = ["English", "Hinglish","Hindi","French","German"]


st.title('LinkedIn Post Generator')

# Create columns for user input
col1, col2, col3 = st.columns(3)
with col1:
    selected_tag = st.selectbox("Select Topic", options=tags)
with col2:
    selected_length = st.selectbox("Select Length", options=length_options)
with col3:
    selected_language = st.selectbox("Select Language", options=language_options)

# Generate post when button is clicked
if st.button("Generate"):
    post = generate_post(selected_tag, selected_length, selected_language)
    st.write(post)
""")

# Now run the Streamlit app using nohup (background execution)
!nohup streamlit run app.py &>/dev/null &

# Install Pyngrok if it's not installed already
# !pip install pyngrok

# Set up ngrok for external access
from pyngrok import ngrok

# Set the ngrok auth token
ngrok.set_auth_token("2rJSBgtMeqqHBmWXXAw4RxlcCoz_Arm2Vt6JQjQyFZFRiXuz")

# Open a tunnel to the Streamlit app running on port 8501
public_url = ngrok.connect(8501)

print(f"Streamlit app is live at {public_url}")

Streamlit app is live at NgrokTunnel: "https://4f7b-35-194-214-9.ngrok-free.app" -> "http://localhost:8501"
